In [2]:
import graphlab as gl

In [3]:
data = gl.SFrame.read_csv('~/CTR/train_subset.csv', verbose=False)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1526306397.log


This non-commercial license of GraphLab Create for academic use is assigned to xiangg@tcd.ie and will expire on May 14, 2019.


In [4]:
data.head(1)

Columns:
	id	float
	click	int
	hour	int
	C1	int
	banner_pos	int
	site_id	str
	site_domain	str
	site_category	str
	app_id	str
	app_domain	str
	app_category	str
	device_id	str
	device_ip	str
	device_model	str
	device_type	int
	device_conn_type	int
	C14	int
	C15	int
	C16	int
	C17	int
	C18	int
	C19	int
	C20	int
	C21	int

Rows: 1

Data:
+-------------+-------+----------+------+------------+----------+-------------+
|      id     | click |   hour   |  C1  | banner_pos | site_id  | site_domain |
+-------------+-------+----------+------+------------+----------+-------------+
| 1.00001e+18 |   0   | 14102100 | 1005 |     0      | 1fbe01fe |   f3845767  |
+-------------+-------+----------+------+------------+----------+-------------+
+---------------+----------+------------+--------------+-----------+-----------+
| site_category |  app_id  | app_domain | app_category | device_id | device_ip |
+---------------+----------+------------+--------------+-----------+-----------+
|    28905ebd   | ecad2386 |  7801e8d9  |   07d7df22   |  a99f214a |  ddd2926e |
+---------------+----------+------------+--------------+-----------+-----------+
+--------------+-------------+------------------+-------+-----+-----+------+-----+
| device_model | device_type | device_conn_type |  C14  | C15 | C16 | C17  | ... |
+--------------+-------------+------------------+-------+-----+-----+------+-----+
|   44956a24   |      1      |        2         | 15706 | 320 |  50 | 1722 | ... |
+--------------+-------------+------------------+-------+-----+-----+------+-----+
[1 rows x 24 columns]

In [5]:
data['click'].mean()

0.17490174901748948

In [6]:
gl.canvas.set_target('ipynb')
data.show()

In [7]:
data.groupby('device_type', {'CTR':gl.aggregate.MEAN('click')})

device_type,CTR
1,0.175977623465
4,0.0725075528701
0,0.227499406317
5,0.0990566037736


In [8]:
data.groupby('C1', {'CTR':gl.aggregate.MEAN('click')})

C1,CTR
1001,0.103448275862
1002,0.227499406317
1007,0.0
1010,0.0742713882795
1008,0.4
1005,0.176174097389


In [9]:
data['C15'].sketch_summary().frequent_items()

{120: 2, 216: 912, 300: 3935, 320: 95132, 728: 18}

In [10]:
data['C16'].sketch_summary().frequent_items()

{20: 2, 36: 912, 50: 95620, 90: 18, 250: 3427, 480: 20}

In [11]:
zip(data.column_names(), data.column_types())

[('id', float),
 ('click', int),
 ('hour', int),
 ('C1', int),
 ('banner_pos', int),
 ('site_id', str),
 ('site_domain', str),
 ('site_category', str),
 ('app_id', str),
 ('app_domain', str),
 ('app_category', str),
 ('device_id', str),
 ('device_ip', str),
 ('device_model', str),
 ('device_type', int),
 ('device_conn_type', int),
 ('C14', int),
 ('C15', int),
 ('C16', int),
 ('C17', int),
 ('C18', int),
 ('C19', int),
 ('C20', int),
 ('C21', int)]

In [12]:
data['device_type'] = data['C1'].astype(str)
data['C1'] = data['C1'].astype(str)

### Step 2   train test split


In [13]:
train_data, test_data = data.random_split(0.8, seed=1)

### Step 3: Selecting a Machine Learning Model

In [14]:
model = gl.logistic_classifier.create(train_data, target='click', features=['device_type', 'C1', 'C15', 'C16'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Logistic regression:

--------------------------------------------------------

Number of examples          : 76199

Number of classes           : 2

...

| 5         | 6        | 1.499911     | 0.823856          | 0.824249            |

| 6         | 7        | 1.592211     | 0.823856          | 0.824249            |

+-----------+----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [15]:
model.predict(test_data, output_type='probability').head(5)

dtype: float
Rows: 5
[0.16564941675062594, 0.2255294552657986, 0.16564941675062594, 0.16564941675062594, 0.16564941675062594]

In [16]:
Y_pred = model.predict(test_data)
Y_pred.head(5)

dtype: int
Rows: 5
[0, 0, 0, 0, 0]

In [24]:
test_data['click'].head(10)

dtype: int
Rows: 10
[1, 0, 0, 0, 0, 0, 1, 0, 0, 1]

In [25]:
Y_test = test_data['click']
Y_test.head(10)

dtype: int
Rows: 10
[1, 0, 0, 0, 0, 0, 1, 0, 0, 1]

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(np.array(Y_test), np.array(Y_pred))

0.82942537989332799

In [28]:
from sklearn.metrics import roc_auc_score
roc_auc_score(np.array(Y_test), np.array(Y_pred))

0.50005659340163544

In [29]:
predict_proba = model.predict(test_data, output_type='probability')
roc_auc_score(np.array(Y_test),np.array(predict_proba))

0.5532955782595973